<a href="https://colab.research.google.com/github/DevNicque/MyCods/blob/master/base_atem_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('atem.csv')
df.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Digite os dados do pesquisado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Qual o cargo do respondente?,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Qual o segmento?,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:",Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,"Em linhas gerais, como considera o relacionamento com o seu assessor comercial:",Unnamed: 38,"SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:",Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Em linhas gerais como considera o serviço de entrega da ATEM?,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,"Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:",Unnamed: 62,Unnamed: 63,"Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14","Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:",Quanto a sua reclamação ou solicitação.,Unnamed: 67,Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:,Unnamed: 69,"(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:",Unnamed: 71,Unnamed: 72,Observações da Pesquisa: (Atendente)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nome,Empresa,Endereço,Endereço 2,Cidade/Município,Estado,Código postal,País,Endereço de email,Número de telefone,Diretor (a),Gerente,Coordenador (a),Assistente,Outro (especifique),ATEM,COF (Consumidor Final),COFG (Consumidor Final Governamental),POSBB (Postos Bandeira Branca),PTAT (Pontão ATEM),PTBB (pontão Bandeira Branca),TRR (Transportador Revendedor Retalista),TRRNI (Transportador Revendedor Retalista na N...,Cordialidade ( é gentil e atencioso nos atendi...,Disponibilidade ( atende fora do dia de visita...,Eficácia ( lhe entrega todas as informações e ...,Eficiência ( traz soluções adequadas para o se...,Orientações Técnicas Legais ( quando necessári...,Response,Por que? comente,Cordialidade do Motorista (ele foi gentil e at...,Entrega no Prazo (o produto foi entregue dentr...,O Ciclo do Pedido é adequado? (tempo decorrido...,Condições de Frota (o estado de manutenção pas...,Procedimentos de segurança no momento da entre...,5.0,4.0,3.0,2.0,1.0,Por que? Comente,Cordialidade do atendimento (o atendimento na ...,Mercadoria disponível no Prazo (o produto esta...,Tempo do ciclo do pedido é adequado? (entre a ...,Procedimentos de segurança no momento da entre...,Há uma comunicação de agenda eficiente?,5.0,4.0,3.0,2.0,1.0,Por que? comente.,Cordialidade ( o atendimento da atendente foi ...,Eficácia ( tempo de espera e tempo de resposta...,Eficiência ( o suporte atendeu suas solicitaçõ...,Response,Response,Recebeu retorno em até 48h?,Foi apresentada solução da sua solicitação?,Response,Por que atribuiu esta nota? Comente,há interesse dos clientes nas ações?,há aumento de vendas no período?,há conquista de novos clientes?,Open-Ended Response
1,1.226668e+10,397988367.0,12/17/2020 01:46:57 PM,12/17/2020 03:05:15 PM,177.149.116.220,NaN,NaN,NaN,NaN,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,NaN,NaN,LUCAS DO RIO,MT,NaN,NaN,NaN,6599936-9977,NaN,Gerente,NaN,NaN,NaN,NaN,NaN,NaN,POSBB (Postos Bandeira Branca),NaN,NaN,NaN,NaN,5,5,5,5,5,5,CLIENTE É BEM ATENDIDO NÃO TEVE PROBLEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,5,5,5,NaN,4.0,NaN,NaN,NaN,AS VEZES ATRASA FORA ISSO OTIMO A

In [40]:
df1 = df.loc[:,'respondent_id':'custom_1']
df1 = df1.drop(0)
df1.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1
1,1.226668e+10,397988367.0,12/17/2020 01:46:57 PM,12/17/2020 03:05:15 PM,177.149.116.220,NaN,NaN,NaN,NaN
2,1.226666e+10,397988367.0,12/17/2020 02:58:55 PM,12/17/2020 02:59:58 PM,177.149.116.220,NaN,NaN,NaN,NaN
3,1.226666e+10,397988367.0,12/17/2020 02:22:27 PM,12/17/2020 02:58:48 PM,177.149.116.220,NaN,NaN,NaN,NaN
4,1.226666e+10,397988367.0,12/17/2020 02:33:53 PM,12/17/2020 02:58:46 PM,177.149.116.220,NaN,NaN,NaN,NaN
5,1.226665e+10,397988367.0,12/17/2020 01:52:50 PM,12/17/2020 02:58:03 PM,177.149.116.220,NaN,NaN,NaN,NaN


In [49]:
df2 = df.loc[:,'Digite os dados do pesquisado':'Unnamed: 18']
df2.columns = df2.loc[0]
df2 = df2.drop(0)

df2.head()

,Nome,Empresa,Endereço,Endereço 2,Cidade/Município,Estado,Código postal,País,Endereço de email,Número de telefone
1,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,NaN,NaN,LUCAS DO RIO,MT,NaN,NaN,NaN,6599936-9977
2,NaN,LEONISIA N N BARROSO,NaN,NaN,CUIABA,MT,NaN,NaN,NaN,(065) 99284-7825
3,NaN,LEONISIA N N BARROSO,NaN,NaN,CUIABA,MT,NaN,NaN,NaN,(065) 99284-7825
4,NaN,LEONISIA N N BARROSO,NaN,NaN,CUIABA,MT,NaN,NaN,NaN,(065) 99284-7825
5,ORGIER,O L QUEIROZ,NaN,NaN,CAROEBE,RORAIMA,NaN,NaN,NaN,(95) 999711717


In [50]:
df3 = df.loc[:,'Qual o cargo do respondente?':'Unnamed: 23']
df3.columns = df3.loc[0]
df3 = df3.drop(0)

df3.head()

,Diretor (a),Gerente,Coordenador (a),Assistente,Outro (especifique)
1,NaN,Gerente,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,Diretor (a),NaN,NaN,NaN,NaN


In [51]:
df4 = df.loc[:,'Qual o segmento?':'Unnamed: 31']
df4.columns = df4.loc[0]
df4 = df4.drop(0)

df4.head()

,ATEM,COF (Consumidor Final),COFG (Consumidor Final Governamental),POSBB (Postos Bandeira Branca),PTAT (Pontão ATEM),PTBB (pontão Bandeira Branca),TRR (Transportador Revendedor Retalista),TRRNI (Transportador Revendedor Retalista na Navegação Interior)
1,NaN,NaN,NaN,POSBB (Postos Bandeira Branca),NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,POSBB (Postos Bandeira Branca),NaN,NaN,NaN,NaN


In [52]:
df5 = df.loc[:,'Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:':'Unnamed: 36']
df5.columns = df5.loc[0]
df5 = df5.drop(0)

df5.head()

,Cordialidade ( é gentil e atencioso nos atendimentos?),Disponibilidade ( atende fora do dia de visita quando necessário?),"Eficácia ( lhe entrega todas as informações e demonstra disponibilidade, em esclarecer suas dúvidas e em negociar?)",Eficiência ( traz soluções adequadas para o seu negócio? otimiza o atendimento?),Orientações Técnicas Legais ( quando necessário lhe passa orientações técnicas?)
1,5,5,5,5,5
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,5,5,5,5,5


In [53]:
df6 = df.loc[:,'Em linhas gerais, como considera o relacionamento com o seu assessor comercial:':'Unnamed: 38']
df6.columns = df6.loc[0]
df6 = df6.drop(0)

df6.head()

,Response,Por que? comente
1,5,CLIENTE É BEM ATENDIDO NÃO TEVE PROBLEMA
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,5,MUITO BOM


In [54]:
df7 = df.loc[:,'SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:':'Unnamed: 43']
df7.columns = df7.loc[0]
df7 = df7.drop(0)

df7.head()

,Cordialidade do Motorista (ele foi gentil e atencioso no atendimento?),Entrega no Prazo (o produto foi entregue dentro do prazo combinado? ),O Ciclo do Pedido é adequado? (tempo decorrido entre a data da realização do pedido e a data efetiva da entrega).,Condições de Frota (o estado de manutenção passa segurança?),"Procedimentos de segurança no momento da entrega? (uso de EPI, uso de Equipamentos de segurança?)"
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN


In [55]:
df8 = df.loc[:,'Em linhas gerais como considera o serviço de entrega da ATEM?':'Unnamed: 49']
df8.columns = df8.loc[0]
df8 = df8.drop(0)

df8.head()

,5.0,4.0,3.0,2.0,1.0,Por que? Comente
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df9 = df.loc[:,'SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:':'Unnamed: 54']
df9.columns = df9.loc[0]
df9 = df9.drop(0)

df9.head()

,Cordialidade do atendimento (o atendimento na base foi gentil e atencioso?),Mercadoria disponível no Prazo (o produto estava pronto para a retirada conforme o acordado?),Tempo do ciclo do pedido é adequado? (entre a compra e a entrega),Procedimentos de segurança no momento da entrega dentro da base,Há uma comunicação de agenda eficiente?
1,5,4,5,5,5
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,5,5,5,5,5


In [60]:
df10 = df.loc[:,'Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?':'Unnamed: 60']
df10.columns = df10.loc[0]
df10 = df10.drop(0)

df10.loc[1:10]

,5.0,4.0,3.0,2.0,1.0,Por que? comente.
1,NaN,4.0,NaN,NaN,NaN,AS VEZES ATRASA FORA ISSO OTIMO ATENDIMENTO
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,1.0,"Sempre tem problemas, atraso constante."
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,1.0,"Sempre tem problemas, atraso constante."
10,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df11 = df.loc[:,'Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:':'Unnamed: 63']
df11.columns = df11.loc[0]
df11 = df11.drop(0)

df11.head()

,Cordialidade ( o atendimento da atendente foi gentil e atencioso?),Eficácia ( tempo de espera e tempo de resposta foram adequados?),Eficiência ( o suporte atendeu suas solicitações/necessidade de forma adequada?)
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,5,5,5


In [34]:
df12 = df.loc[:,'Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14']
df12.columns = df12.loc[0]
df12 = df12.drop(0)

df12.head()

0    Response
1         NÃO
2         NaN
3         NaN
4         NaN
Name: Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14, dtype: object

In [69]:
df13 = df['Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:']
df13.columns = df13.loc[0]
df13 = df13.drop(0)

df13.head()

1                  NaN
2                  NaN
3                  NaN
4                  NaN
5    Entre 2 e 5 vezes
Name: Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:, dtype: object

In [28]:
df14 = df.loc[:,'Quanto a sua reclamação ou solicitação.':'Unnamed: 67']
df14.head()

,Quanto a sua reclamação ou solicitação.,Unnamed: 67
0,Recebeu retorno em até 48h?,Foi apresentada solução da sua solicitação?
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [26]:
df15 = df.loc[:,'Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:':'Unnamed: 69']
df15.head()

,Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:,Unnamed: 69
0,Response,Por que atribuiu esta nota? Comente
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [25]:
df16 = df.loc[:,'(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:':'Unnamed: 72']
df16.head()

,"(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:",Unnamed: 71,Unnamed: 72
0,há interesse dos clientes nas ações?,há aumento de vendas no período?,há conquista de novos clientes?
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [37]:
df17 = df.loc[:,'Observações da Pesquisa: (Atendente)']
df17.head()

0                        Open-Ended Response
1                                  MIKAELLEN
2    Cliente não tem interesse em responder.
3    Cliente não tem interesse em responder.
4    Cliente não tem interesse em responder.
Name: Observações da Pesquisa: (Atendente), dtype: object